# High-throughput screening for novel materials? - Exercises

In this notebook, you find some exercises related with how to perform a high-throughput screening for light harvesting materials.

Use the below cell to read the database into memory.

In [ ]:
%matplotlib inline
from ase.db import connect
con = connect('Lecture_6/cubic_perovskites.db')

## Exercise 1
From the cubic_perovsites.db database, identify the oxides that can be used for one photon water splitting device.

You can take inpiration from the CMR database, https://cmr.fysik.dtu.dk/cubic_perovskites/cubic_perovskites.html#cubic-perovskites.

In [ ]:
# teacher
from ase.phasediagram import PhaseDiagram

references = [(row.formula, row.energy)
              for row in con.select('reference')]

for row in con.select(combination='ABO3'):
    pd = PhaseDiagram(references, filter=row.formula, verbose=False)
    energy = pd.decompose(row.formula)[0]
    heat = (row.energy - energy) / row.natoms
    if (heat < 0.21 and
        (3.1 > row.gllbsc_ind_gap > 1.4 or
         3.1 > row.gllbsc_dir_gap > 1.4) and
        (row.VB_ind - 4.5 > 1.23 and row.CB_ind - 4.5 < 0 or
         row.VB_dir - 4.5 > 1.23 and row.CB_dir - 4.5 < 0)):
        formula = row.A_ion + row.B_ion + row.anion
        print('{0}, {1}, {2:.3f}'.format(row.id, formula, heat))

## Exercise 2
Use AgNbO$_3$, and calculate the convex hull and Pourbaix diagram using data from the database.
See this tutorial for inspiration: https://wiki.fysik.dtu.dk/ase/ase/phasediagram/phasediagram.html

For the Pourbaix diagram, remember to give the reference energies with respect to the standard states.

In [ ]:
# teacher
from ase.phasediagram import PhaseDiagram
import matplotlib.pyplot as plt

references = [(row.formula, row.energy)
              for row in con.select('reference')]

for row in con.select(formula='AgNbO3', A_ion='Ag'):
    atoms = row.toatoms()
    references += [('AgNbO3', row.energy)]
    pd = PhaseDiagram(references, filter=row.formula, verbose=False)
    energy = pd.decompose(row.formula)[0]
    fig, ax = plt.subplots()
    pd.plot(ax=ax, show=False)
    plt.title('A-ion:{} B-ion: {}'.format(row.A_ion, row.B_ion))
plt.show()

Is the material on the convex hull? If not, why is it still a candidate?

In [ ]:
# teacher
from ase.phasediagram import Pourbaix, solvated
import numpy as np

unique = ['Ag','Nb','O']
refsd = {}
for row in con.select():
    isref = True
    for j in row.symbols:
        if j not in unique:
            isref = False
    if isref:
        refsd[row.formula] = row.energy
        print(row.formula, row.energy)

refs = solvated('AgNb')
refs += [('Ag', 0),
         ('Nb', 0),
         ('Ag2O3', refsd['Ag2O3'] - 2 * refsd['Ag4'] / 4 - 3 * refsd['O']),
         ('Nb2O3', refsd['Nb2O3'] - 2 * refsd['Nb2'] / 2 - 3 * refsd['O']),
         ('Ag4O2', refsd['Ag4O2'] - 4 * refsd['Ag4'] / 4 - 2 * refsd['O']),
         ('Nb16O32', refsd['Nb16O32'] - 16 * refsd['Nb2'] / 2 - 32 * refsd['O']),
        ]

pb = Pourbaix(refs, Ag=1, Nb=1, O=3)

U = np.linspace(-2, 2, 200)
pH = np.linspace(-2, 16, 300)
d, names, text = pb.diagram(U, pH, plot=True)

## Exercise 3
One of the possible solutions to instability in water is to add a transparent protective layer with appropriate position of the bands and stable in water. Find the possible transparent layers in the oxo-perovskites for AgNbO$_3$. (hint: consider only the oxygen evolution band edge and, for simplicity, consider only the indirect gap and discard the calculation of Pourbax diagrams)

In [ ]:
# teacher
from ase.phasediagram import PhaseDiagram

for row in con.select(formula='AgNbO3', A_ion='Ag'):
    VB_ind = row.VB_ind
    CB_ind = row.CB_ind

references = [(row.formula, row.energy)
              for row in con.select('reference')]

for row in con.select(combination='ABO3'):
    pd = PhaseDiagram(references, filter=row.formula, verbose=False)
    energy = pd.decompose(row.formula)[0]
    heat = (row.energy - energy) / row.natoms
    if (heat < 0.21 and
        (row.gllbsc_ind_gap > 3.1) and
        (row.VB_ind - 4.5 > 1.23 and row.VB_ind < VB_ind)):
        formula = row.A_ion + row.B_ion + row.anion
        print('{0}, {1}, {2:.3f}'.format(row.id, formula, heat))

Did you find any good candidate? If not, try with another material.

## Exercise 4
Relax, calculate the band gap and band structure at the GLLB-SC level of a possible new candidate structure: CaTiO$_3$ and compare it with SrTiO$_3$. Take inspiration from the exercises of the previous lecture and https://wiki.fysik.dtu.dk/gpaw/tutorials/band_gap/band_gap.html and https://wiki.fysik.dtu.dk/gpaw/exercises/band_structure/bands.html

Calculate the band structure and band gap of both structures.

The GLLB functional contains a bug, where it is unable to parallelize over bands, i.e. if you use plane-wave mode. You can avoid this error by using the following keyword in your GPAW calculator
```python
parallel={'band': 1}
```

As we will need to plot the band structure later, make sure that you save all of the wave functions for your fixed density calculation, i.e. use `mode='all'` when writing your wave functions:
```python
calc.write('myfile.gpw', mode='all')
```
We have provided you with a function to create a band structure object, which  you can plot using something along the lines of
```python
gap, bs = get_band_structure('myfile.gpw')
print('Gap:', gap)
bs.plot()
```

In [ ]:
%%writefile relax.py
# teacher
import numpy as np
from ase import Atoms
from ase.db import connect
from ase.optimize import BFGS
from ase.constraints import StrainFilter
from ase.dft.bandgap import bandgap
from ase.parallel import parprint, paropen
from gpaw import GPAW, FermiDirac, PW

gaps = {}

for name in ['SrTiO3', 'CaTiO3']:
    parprint('Running:', name)
    a0 = 3.8
    atoms = Atoms(symbols=name,
                 pbc=True,
                 cell=np.array([[ a0,  0.,  0.],
                                [ 0.,  a0,  0.],
                                [ 0.,  0.,  a0]]),
                 positions=np.array([[ 0.,  0.,  0.],
                                     [ a0/2,  a0/2,  a0/2],
                                     [ a0/2,  a0/2,  0.],
                                     [ 0.,  a0/2,  a0/2],
                                     [ a0/2,  0.,  a0/2]]))

    # Optimize using PBEsol
    calc = GPAW(mode=PW(600),
                kpts={'size': (11, 11, 11), 'gamma': True},
                xc='PBEsol',
                occupations=FermiDirac(width=0.05),
                txt=name+'_calculator_pbesol.out')
    atoms.set_calculator(calc)

    sf = StrainFilter(atoms)
    dyn = BFGS(sf, trajectory=name+'_pbesol.traj', logfile=name+'_relax.log')
    dyn.run(fmax=0.02)
    calc.write(name+'_pbesol_gs.gpw')  # Store .gpw file for later use

    # Calculate the band structure
    calc = GPAW(name+'_pbesol_gs.gpw',
                fixdensity=True,
                symmetry='off',
                kpts={'path': 'GXMGRX', 'npoints': 80},
                occupations=FermiDirac(width=0.05),
                txt=name+'_calculator_pbesol_bs.out')

    atoms.set_calculator(calc)
    atoms.get_potential_energy()  # Run calculation

    calc.write(name+'_pbesol_bs.gpw', mode='all')  # Store .gpw
    
    # Get PBEsol bandgap
    parprint('PBEsol gap')
    gap_pbesol, p1, p2 = bandgap(calc)

    # Get gap using GLLB
    # Fails with band parallelisation, so we use grid mode
    # and we need a clean calculator object
    calc = GPAW(mode=PW(600),
                parallel={'band': 1},
                kpts={'size': (11, 11, 11), 'gamma': True},
                xc='GLLBSC',
                occupations=FermiDirac(width=0.05),
                txt=name+'_calculator_gllb_gs.out')

    atoms.set_calculator(calc)
    atoms.get_potential_energy()  # Run calculation

    calc.write(name+'_gllb_gs.gpw')  # Store .gpw

    # Get GLLBsc band gap
    response = calc.hamiltonian.xc.xcs['RESPONSE']
    response.calculate_delta_xc()
    EKs, Dxc = response.calculate_delta_xc_perturbation()
    # fundamental band gap
    # EKs = kohn-sham bandgap
    # Dxc = derivative discontinuity
    gap_gllb = EKs + Dxc

    parprint("Calculated band gap with GLLBsc: {:.3f}".format(gap_gllb))
    
    # Get the band structure
    calc = GPAW(name+'_gllb_gs.gpw',
                parallel={'band': 1},
                fixdensity=True,
                symmetry='off',
                kpts={'path': 'GXMGRX', 'npoints': 80},
                txt=name+'_calculator_gllb_bs.out')

    atoms.set_calculator(calc)
    atoms.get_potential_energy()  # Run calculation
    
    # Get GLLBsc band gap
    response = calc.hamiltonian.xc.xcs['RESPONSE']
    response.calculate_delta_xc()
    EKs, Dxc = response.calculate_delta_xc_perturbation()
    
    calc.write(name+'_gllb_bs.gpw', mode='all')  # Store .gpw
    
    gaps[name] = (gap_pbesol, gap_gllb, EKs, Dxc)

with paropen('results.txt', 'w') as f:
    for name, (gap_pbesol, gap_gllb, EKs, Dxc) in gaps.items():
        print('Name: {}'.format(name), file=f)
        print('PBEsol gap: {:.3f} ev'.format(gap_pbesol), file=f)
        print('GLLBsc gap: {:.3f} eV'.format(gap_gllb), file=f)
        print('EKs: {:.3f} eV\nDxc: {:.3f} eV'.format(EKs, Dxc), file=f)


In [ ]:
# teacher
!bsub.py -p 20 -t 2 relax.py

In [ ]:
# teacher
!cat results.txt

In [ ]:
from gpaw import GPAW
from ase.dft.bandgap import bandgap

def get_band_structure(gpw):
    '''Get a band structure object from a .gpw file.
    Note that wave functions need to be saved to the .gpw file to work.
    I.e. in your calculation, do
    calc.write('myfile.gpw', mode='all')
    
    Then plot the band structure with something like
    
    gap, bs = get_band_structure('myfile.gpw')
    print('Gap:', gap)
    bs.plot()
    '''
    calc = GPAW(gpw, txt=None)
    bs = calc.band_structure()
    gap = bandgap(calc)[0]
    xc = calc.get_xc_functional()
    ef = calc.get_fermi_level()
    if xc == 'GLLBSC':
        # Get GLLBsc band gap
        response = calc.hamiltonian.xc.xcs['RESPONSE']
        response.calculate_delta_xc()
        EKs, Dxc = response.calculate_delta_xc_perturbation()
        homo, lumo = calc.get_homo_lumo()
        # Modify energies
        bs.energies[bs.energies >= ef] += Dxc
        gap += Dxc
        
    bs.energies -= ef  # Center energies at Fermi level
    bs.reference -= ef
    return gap, bs

In [ ]:
# teacher
from glob import glob
from IPython.utils import io

structs = ['SrTiO3', 'CaTiO3']
funcs = ['pbesol', 'gllb']

bands = {}
# unfortunately, gllb always prints a lot of info
# So we just capture it, just because it's annoying
with io.capture_output() as captured:
    for name in structs:
        for xc in funcs:
            gpw = '{}_{}_bs.gpw'.format(name, xc)
            gap, bs = get_band_structure(gpw)
            key = '{}--{}'.format(name, xc)
            bands[key] = (gap, bs)

In [ ]:
# teacher
import matplotlib.pyplot as plt


f, axes = plt.subplots(2, 2, figsize=(15, 10))
for ii, (key, value) in enumerate(bands.items()):
    gap, bs = value
    name, xc = key.split('--')
    i, j = divmod(ii, 2)
    ax = axes[i, j]
    bs.plot(ax=ax, show=False)
    ax.set_title('Name: {}, xc: {}, gap: {:.3f} eV'.format(name, xc, gap))
plt.show()